In [ ]:
# Run this cell to install required packages
%pip install segment-geospatial geopandas folium ipywidgets google-auth-oauthlib google-auth-httplib2 google-api-python-client matplotlib descartes rasterio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 75.6 M

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import csv
import torch
import torch.nn.functional as F
from torchvision import transforms
from samgeo import tms_to_geotiff
from PIL import Image
# import matplotlib.pyplot as plt  # Optional if you don't need to display images

def download_and_resize_image(image_path, bbox, zoom, source, desired_size):
    try:
        # Download the image using the geographic bounding box
        tms_to_geotiff(output=image_path, bbox=bbox, zoom=zoom, source=source, overwrite=True)

        # Open the downloaded image
        with Image.open(image_path) as img:
            original_size = img.size

            # Convert PIL image to PyTorch tensor
            transform_to_tensor = transforms.ToTensor()
            img_tensor = transform_to_tensor(img)

            # Add batch dimension
            img_tensor = img_tensor.unsqueeze(0)

            # Move tensor to GPU if available
            if torch.cuda.is_available():
                device = torch.device('cuda')
                img_tensor = img_tensor.to(device)
            else:
                print("CUDA is not available. Using CPU.")
                device = torch.device('cpu')

            # Resize the image using torch.nn.functional.interpolate
            desired_size_pytorch = (desired_size[1], desired_size[0])  # (height, width)
            resized_tensor = F.interpolate(img_tensor, size=desired_size_pytorch, mode='bilinear', align_corners=False)

            # Move tensor back to CPU if necessary
            if device.type == 'cuda':
                resized_tensor = resized_tensor.cpu()

            # Remove batch dimension
            resized_tensor = resized_tensor.squeeze(0)

            # Convert tensor back to PIL image
            transform_to_pil = transforms.ToPILImage()
            img_resized = transform_to_pil(resized_tensor)

            # Display the image (optional)
            # plt.imshow(img_resized)
            # plt.axis('off')  # Hide axes
            # plt.show()

            return original_size, img_resized.size, img_resized
    except Exception as e:
        print(f"Failed to download or resize image for bbox {bbox}. Error: {e}")
        return None, None, None

def process_csv(file_path, save_directory):
    zoom = 22
    desired_image_size = (512, 512)  # Desired image dimensions (width, height)

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Read the header row

        for row_num, row in enumerate(csv_reader, start=1):  # Process each row from the CSV
            bbox = list(map(float, row[1:5]))  # Assuming bbox columns are second to fifth
            filename = row[0]  # Assuming the filename is in the first column

            # Create full save path
            image_path = os.path.join(save_directory, filename)

            # Check if the image already exists
            if os.path.exists(image_path):
                print(f"Image {filename} already exists in {save_directory}. Skipping.")
                continue

            # Ensure the directory exists
            os.makedirs(save_directory, exist_ok=True)

            # Download and resize the image
            original_size, resized_size, img_resized = download_and_resize_image(
                image_path, bbox, zoom, "Satellite", desired_image_size
            )

            if img_resized:
                img_resized.save(image_path)
                print(f"Image saved as {image_path}.")
            else:
                print(f"Failed to process image for row {row_num}.")

if __name__ == "__main__":
    # Path to the CSV file and directory to save patches
    csv_file_path = r'/content/drive/MyDrive/Madhu RA Work Folder/xl/patches_with_multi_buildings_illinois.csv'
    save_directory = r'/content/drive/MyDrive/Madhu RA Work Folder/Img/Multi'

    # Call the function to process the CSV and download the images
    process_csv(csv_file_path, save_directory)


Streaming output truncated to the last 5000 lines.
Downloaded image 1476/2107
Downloaded image 1477/2107
Downloaded image 1478/2107
Downloaded image 1479/2107
Downloaded image 1480/2107
Downloaded image 1481/2107
Downloaded image 1482/2107
Downloaded image 1483/2107
Downloaded image 1484/2107
Downloaded image 1485/2107
Downloaded image 1486/2107
Downloaded image 1487/2107
Downloaded image 1488/2107
Downloaded image 1489/2107
Downloaded image 1490/2107
Downloaded image 1491/2107
Downloaded image 1492/2107
Downloaded image 1493/2107
Downloaded image 1494/2107
Downloaded image 1495/2107
Downloaded image 1496/2107
Downloaded image 1497/2107
Downloaded image 1498/2107
Downloaded image 1499/2107
Downloaded image 1500/2107
Downloaded image 1501/2107
Downloaded image 1502/2107
Downloaded image 1503/2107
Downloaded image 1504/2107
Downloaded image 1505/2107
Downloaded image 1506/2107
Downloaded image 1507/2107
Downloaded image 1508/2107
Downloaded image 1509/2107
Downloaded image 1510/2107
Down

In [ ]:
import os
import csv
import torch
import torch.nn.functional as F
from torchvision import transforms
from samgeo import tms_to_geotiff
from PIL import Image
# import matplotlib.pyplot as plt  # Optional if you don't need to display images

def download_and_resize_image(image_path, bbox, zoom, source, desired_size):
    try:
        # Download the image using the geographic bounding box
        tms_to_geotiff(output=image_path, bbox=bbox, zoom=zoom, source=source, overwrite=True)

        # Open the downloaded image
        with Image.open(image_path) as img:
            original_size = img.size

            # Convert PIL image to PyTorch tensor
            transform_to_tensor = transforms.ToTensor()
            img_tensor = transform_to_tensor(img)

            # Add batch dimension
            img_tensor = img_tensor.unsqueeze(0)

            # Move tensor to GPU if available
            if torch.cuda.is_available():
                device = torch.device('cuda')
                img_tensor = img_tensor.to(device)
            else:
                print("CUDA is not available. Using CPU.")
                device = torch.device('cpu')

            # Resize the image using torch.nn.functional.interpolate
            desired_size_pytorch = (desired_size[1], desired_size[0])  # (height, width)
            resized_tensor = F.interpolate(img_tensor, size=desired_size_pytorch, mode='bilinear', align_corners=False)

            # Move tensor back to CPU if necessary
            if device.type == 'cuda':
                resized_tensor = resized_tensor.cpu()

            # Remove batch dimension
            resized_tensor = resized_tensor.squeeze(0)

            # Convert tensor back to PIL image
            transform_to_pil = transforms.ToPILImage()
            img_resized = transform_to_pil(resized_tensor)

            # Display the image (optional)
            # plt.imshow(img_resized)
            # plt.axis('off')  # Hide axes
            # plt.show()

            return original_size, img_resized.size, img_resized
    except Exception as e:
        print(f"Failed to download or resize image for bbox {bbox}. Error: {e}")
        return None, None, None

def process_csv(file_path, save_directory):
    zoom = 22
    desired_image_size = (512, 512)  # Desired image dimensions (width, height)

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Read the header row

        for row_num, row in enumerate(csv_reader, start=1):  # Process each row from the CSV
            bbox = list(map(float, row[1:5]))  # Assuming bbox columns are second to fifth
            filename = row[0]  # Assuming the filename is in the first column

            # Create full save path
            image_path = os.path.join(save_directory, filename)

            # Check if the image already exists
            if os.path.exists(image_path):
                print(f"Image {filename} already exists in {save_directory}. Skipping.")
                continue

            # Ensure the directory exists
            os.makedirs(save_directory, exist_ok=True)

            # Download and resize the image
            original_size, resized_size, img_resized = download_and_resize_image(
                image_path, bbox, zoom, "Satellite", desired_image_size
            )

            if img_resized:
                img_resized.save(image_path)
                print(f"Image saved as {image_path}.")
            else:
                print(f"Failed to process image for row {row_num}.")

if __name__ == "__main__":
    # Path to the CSV file and directory to save patches
    csv_file_path = r'/content/drive/MyDrive/Madhu RA Work Folder/xl/patches_with_multi_buildings_illinois.csv'
    save_directory = r'/content/drive/MyDrive/Madhu RA Work Folder/Img/multi'

    # Call the function to process the CSV and download the images
    process_csv(csv_file_path, save_directory)

Streaming output truncated to the last 5000 lines.
Downloaded image 1383/2107
Downloaded image 1384/2107
Downloaded image 1385/2107
Downloaded image 1386/2107
Downloaded image 1387/2107
Downloaded image 1388/2107
Downloaded image 1389/2107
Downloaded image 1390/2107
Downloaded image 1391/2107
Downloaded image 1392/2107
Downloaded image 1393/2107
Downloaded image 1394/2107
Downloaded image 1395/2107
Downloaded image 1396/2107
Downloaded image 1397/2107
Downloaded image 1398/2107
Downloaded image 1399/2107
Downloaded image 1400/2107
Downloaded image 1401/2107
Downloaded image 1402/2107
Downloaded image 1403/2107
Downloaded image 1404/2107
Downloaded image 1405/2107
Downloaded image 1406/2107
Downloaded image 1407/2107
Downloaded image 1408/2107
Downloaded image 1409/2107
Downloaded image 1410/2107
Downloaded image 1411/2107
Downloaded image 1412/2107
Downloaded image 1413/2107
Downloaded image 1414/2107
Downloaded image 1415/2107
Downloaded image 1416/2107
Downloaded image 1417/2107
Down